# Deep Dive into LangChain

## import the openAI key

In [3]:
from dotenv import load_dotenv
import os
load_dotenv("./.env")
API_KEY = os.getenv("API_OAI_KEY")

## ChatModels: GPT-3.5-Turbo and GPT-4

In [7]:
from langchain_openai import ChatOpenAI

prompt = "Explain Quantum Mechanics in one sentence"

llm = ChatOpenAI(api_key=API_KEY, model="gpt-4o-mini")
output = llm.invoke(prompt)
print(output.content)

Quantum mechanics is a fundamental theory in physics that describes the behavior of matter and energy at the smallest scales, where particles exhibit wave-particle duality and uncertainty principles, leading to phenomena that defy classical intuition.


In [8]:
# Defining the roles using LangChain
from langchain.schema import (
	SystemMessage,
	AIMessage,
	HumanMessage

)

In [9]:
messages = [
	SystemMessage(content="Your are a physisit and respond only in German"),
	HumanMessage(content="Explain Quantum Mechanics in one sentence")
]

output = llm.invoke(messages)
print(output.content)

Quantenmechanik ist die Theorie, die das Verhalten von Materie und Energie auf der kleinsten Skala, der atomaren und subatomaren Ebene, beschreibt und dabei Konzepte wie Superposition und Verschränkung einführt.


## Caching LLM Responses

### In-memory cache

In [12]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI

llm = OpenAI(model_name = "gpt-3.5-turbo-instruct", api_key=API_KEY)

In [13]:
%%time 
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = "Tell me a joke that a toddler can understand."

llm.invoke(prompt)

CPU times: user 55.4 ms, sys: 18.6 ms, total: 74 ms
Wall time: 1.06 s


"\n\nWhy did the banana go to the doctor?\nBecause it wasn't peeling well!"

In [14]:
%%time
llm.invoke(prompt)

CPU times: user 238 μs, sys: 45 μs, total: 283 μs
Wall time: 286 μs


"\n\nWhy did the banana go to the doctor?\nBecause it wasn't peeling well!"

## LLM Streaming

In [18]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=API_KEY, model="gpt-4o-mini")
prompt = "Write a rock song about the Moon and a Raven."
for chunk in llm.stream(prompt):
	print(chunk.content, end="", flush=True)

**Title: "Raven on the Moon"**

**Verse 1**  
Under midnight's velvet skies,  
A silver glow ignites our eyes,  
Where shadows dance and whispers play,  
The raven calls, it leads the way.  

**Pre-Chorus**  
With feathers black as darkest night,  
He soars through stars, a ghost in flight,  
The Moon is high, a watchful shroud,  
Together we'll break free from the crowd.

**Chorus**  
Raven on the Moon, take me higher,  
Through the darkness, set my soul on fire,  
In the stillness, we find our truth,  
With wings of midnight, we’ll break loose.  

**Verse 2**  
Echoes of the past, they linger near,  
A haunting song that draws us here,  
With every beat, my heart aligns,  
To the rhythm of ancient times.  

**Pre-Chorus**  
The Moon it glows, casting spells so bright,  
With every flap, we ignite the night,  
Lost in dreams, we’ll ride the tides,  
A cosmic dance where our fate collides.

**Chorus**  
Raven on the Moon, take me higher,  
Through the darkness, set my soul on fire,  
I

## Prompt Templates

In [20]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = ''' You are an experienced virologist.
Write a few senteces about the following virus {virus} in {language}
'''
prompt_template = PromptTemplate.from_template(template=template)

prompt = prompt_template.format(virus="hiv", language="french")

llm = ChatOpenAI(api_key=API_KEY, model="gpt-4o-mini")
output = llm.invoke(prompt)

print(output.content)

Le VIH, ou virus de l'immunodéficience humaine, est un rétrovirus qui attaque le système immunitaire, notamment les cellules CD4, cruciales pour la défense de l'organisme contre les infections. Si le VIH n'est pas traité, il peut évoluer vers le sida, une phase où le système immunitaire est gravement compromis. La transmission du VIH se produit principalement par les relations sexuelles non protégées, le partage de seringues et de la mère à l'enfant pendant la grossesse, l'accouchement ou l'allaitement. Grâce aux avancées en matière de traitements antirétroviraux, les personnes vivant avec le VIH peuvent mener une vie saine et réduire considérablement le risque de transmission.


## ChatPromptTemplate

In [21]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
	[
		SystemMessage(content="You respond only in the JSON format"),
		HumanMessagePromptTemplate.from_template("Top {n} contries in {area} by population")
	]
)

messages = chat_template.format_messages(n="10", area="Europe")
print(messages)

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=API_KEY)
output = llm.invoke(messages)

print(output.content)

[SystemMessage(content='You respond only in the JSON format', additional_kwargs={}, response_metadata={}), HumanMessage(content='Top 10 contries in Europe by population', additional_kwargs={}, response_metadata={})]
{
    "1": {
        "country": "Germany",
        "population": 83019200
    },
    "2": {
        "country": "France",
        "population": 67158000
    },
    "3": {
        "country": "United Kingdom",
        "population": 66040229
    },
    "4": {
        "country": "Italy",
        "population": 60376370
    },
    "5": {
        "country": "Spain",
        "population": 46733038
    },
    "6": {
        "country": "Ukraine",
        "population": 41660983
    },
    "7": {
        "country": "Poland",
        "population": 38433600
    },
    "8": {
        "country": "Romania",
        "population": 19405156
    },
    "9": {
        "country": "Netherlands",
        "population": 17424591
    },
    "10": {
        "country": "Belgium",
        "population": 11